In [ ]:
%pip install transformers


In [2]:
%pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [3]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00


In [4]:
%pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Code organization:
- Imports
- Datasets
- Models
    - Question answering
    - Summarization
- Loss function
    - Question answering
    - Summarization
- Out of the box model use
    - preprocessing
    - postprocessing and metrics
- Fine tuned models
    - Preprocessing
    - Training
    - Validation
    - Testing
    - Metrics
- Comparison

## Imports

In [5]:
from transformers import pipeline
import torch
import os
# import accelerate


In [6]:
from datasets import load_dataset
import datasets
import pandas as pd
from torch.nn import CrossEntropyLoss
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import evaluate
from transformers import AutoModelForQuestionAnswering, AutoTokenizer
from transformers import AutoModelForSeq2SeqLM

## Datasets

In [22]:
trainpath = "projekt_data/train.jsonl"
testpath = "projekt_data/test.jsonl"
validpath = "projekt_data/validation.jsonl"

In [23]:
train_dataset = load_dataset("json", data_files=trainpath, split="train")

In [24]:
valid_dataset = load_dataset("json", data_files=validpath, split="train")

In [25]:
test_dataset = load_dataset("json", data_files=testpath, split="train")
#test_dataset = valid_dataset["train"]

In [12]:
test_dataset["spoilerPositions"][0]

[[[4, 188], [4, 192]]]

In [13]:
type(test_dataset)

datasets.arrow_dataset.Dataset

Umjesto da budu svi ovi nepotrebni stupci, ukloniti nepotrebne, zadržati TargetTitle = questions, PostText = drugi dio questions ako budem imala energije, TargetParagraphs = context, spoiler = answer i spoiler_positions. Spoiler_positions treba reindeksirati tako da bude bolje.

In [14]:
# [[[4, 188], [4, 192]]]
def recalculate_indices(context_paragraphs, answer, answer_positions):

    # Combine context paragraphs into a single string
    combined_context = " ".join(context_paragraphs)

    # Find the answer within the combined context
    answer_start = combined_context.find(answer)

    # Check if answer is found
    if answer_start == -1:
        answer_start = 0
        # raise ValueError("Answer not found in the combined context")

    # Calculate character offset based on paragraph indices
    char_offset = 0
    #[[[4, 188], [4, 192]]]

    start_paragraph = answer_positions[0][0][0]
    start_index = answer_positions[0][0][1]
    for i in range(0, start_paragraph): # za svaki pocetni paragraf, answer_positions = [[[start_par, start_index], [end_par, end_index]]]
        char_offset += len(context_paragraphs[i]) + 1  # Add 1 for space between paragraphs

    # Recalculate start and end indices based on answer position and offset
    start_index = answer_start + char_offset + start_index # zar nije index samo answer_start ili cshr_offset + answer_positions
    end_index = start_index + len(answer)

    return (start_index, end_index)

In [20]:
def clean_dataset(dataset):
  start_positions = []
  end_positions = []

  # remove multi answer entries and preprocess
  dataset = dataset.filter(lambda x: "multi" not in x["tags"] and x["targetTitle"] != "" and x["targetParagraphs"] != [])
  contexts = []
  start_positions = []
  end_positions = []
  for paragraphs, answer, answer_positions in zip(dataset['targetParagraphs'], dataset['targetTitle'], dataset["spoilerPositions"]):
        # reindex from [paragraph, index_in_paragraph] to index in total context
        # as context needs to be one string and not a list of strings
        start_index, end_index = recalculate_indices(paragraphs, answer, answer_positions)
        start_positions.append(start_index)
        end_positions.append(end_index)

        # join context as it can't be a list of strings/paragrapgh but just one string
        sentence = []
        for i, paragraph in enumerate(paragraphs):
            sentence.append(str(i) + ": " + paragraph)
        contexts.append(" ".join(sentence))


  new_dataset = dataset.select_columns(['uuid', 'targetTitle', 'spoiler', 'tags'])

  new_dataset = new_dataset.rename_columns({'uuid': 'id', 'targetTitle': 'question', 'spoiler': 'answers'})
  print(type(new_dataset))
  dset_embed = datasets.Dataset.from_dict({"context":contexts, "start_positions":start_positions, "end_positions":end_positions})
  new_dataset = datasets.concatenate_datasets([new_dataset, dset_embed], axis=1)
  return new_dataset


In [26]:
train_dataset = clean_dataset(train_dataset)
valid_dataset = clean_dataset(valid_dataset)

<class 'datasets.arrow_dataset.Dataset'>


Flattening the indices:   0%|          | 0/2640 [00:00<?, ? examples/s]

<class 'datasets.arrow_dataset.Dataset'>


Flattening the indices:   0%|          | 0/656 [00:00<?, ? examples/s]

In [27]:
test_dataset = clean_dataset(test_dataset)

<class 'datasets.arrow_dataset.Dataset'>


Flattening the indices:   0%|          | 0/825 [00:00<?, ? examples/s]

In [28]:
print(test_dataset["question"])

['The Simple Household Item This 24-Year-Old Is Using To Change The World', "You'll Never Believe Who Robert Downey Jr.'s 'Free Pass' Is!", "The Newest Theory on Hiddleswift's Romance Is Bizarre but Strangely Compelling", 'The Little Girl From "Titanic" Is All Grown Up And Still Crazy\xa0Adorable', 'Why we never really get over that first love', 'You Should NEVER Line The Toilet Seat With Paper. The Reason? SHOCKIN...', 'She Puts Chicken Nuggets on the Waffle Iron. What She Does Next Will Blow Your Mind!', 'Ratan Tata Says He Is Lonely, Having A Painful Time Amid Ongoing Tussle With Cyrus Mistry', "Here's why I gave up my beloved Galaxy S7 for a boring old iPhone", 'BREAKING: Loch Ness Monster Found Dead', "There's a New Best City in America for Startups (Hint: It's Not Silicon Valley)", "40 People Got Burned 'Firewalking' in Texas For the Dumbest Reason", 'Phoenix Cancel iTunes Festival Performance After Thomas Mars Falls Sick', 'Bill Clinton delivers a sharp message to Donald Trump a

In [47]:
datasets= {"train": train_dataset, "validation": valid_dataset, "test": test_dataset}

## Models:

Instantiating all the models we're trying out

### Question answering models

For QA models, input should be in form:
```json
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
```

In [19]:
qa_model_names = ["deepset/roberta-base-squad2", "deepset/tinyroberta-squad2", "timpal0l/mdeberta-v3-base-squad2"]# , "facebook/bart-large-cnn", ]

qa_pipelines = []
qa_models = []
qa_tokenizers = []


for model_name in qa_model_names:
    qa_pipelines.append(pipeline('question-answering', model=model_name, tokenizer=model_name))
    qa_models.append(AutoModelForQuestionAnswering.from_pretrained(model_name))
    qa_tokenizers.append(AutoTokenizer.from_pretrained(model_name))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/453 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

### Summarization models

For summarization models, input should be in form:
```json
Sum_input = """...text... """
```

In [ ]:
sum_model_names = ["facebook/bart-large-cnn", "Falconsai/text_summarization", ] #, "google/pegasus-xsum", ]
sum_pipelines = []
sum_models = []
sum_tokenizers = []
for model_name in sum_model_names:
    sum_pipelines.append(pipeline('summarization', model=model_name, tokenizer=model_name))
    sum_models.append(AutoModelForSeq2SeqLM.from_pretrained(model_name))
    sum_tokenizers.append(AutoTokenizer.from_pretrained(model_name))

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.84G [00:00<?, ?B/s]

## Loss functions

### Question answering loss

In [54]:
def qa_loss(start_logits, end_logits, start_positions, end_positions):
    """
    Compute the question answering loss.
    """
    start_loss = CrossEntropyLoss(start_logits, start_positions)
    end_loss = CrossEntropyLoss(end_logits, end_positions)
    total_loss = (start_loss + end_loss) / 2
    return total_loss

### Summarization loss

In [ ]:
# TODO
def sum_loss():
    pass

## Out-of-box model use

### Question answering

Pipeline to evaluate test dataset on QA model

In [20]:
tp = 0
tn = 0
fp = 0
fn = 0

dataloader = DataLoader(test_dataset, batch_size=32, shuffle = True)
evaluator = evaluate.evaluator("question-answering")
metrics = evaluate.combine(["exact_match", "f1"])

for batch in dataloader:
    predicted = qa_pipelines[1](question=batch["question"],
                                context=batch["context"])
    predictions = pd.DataFrame(predicted)
    references = batch["answers"][0]

    # Ensure references and predictions are lists of strings
    references = [str(ref) for ref in references]
    predictions = [str(pred) for pred in predictions["answers"]]

    # Add batch to metrics
    try:
        metrics.add_batch(references=references, predictions=predictions)
        print("Batch added successfully")
    except ValueError as e:
        print("ValueError encountered:", e)
    except Exception as e:
        print("An error occurred:", e)

metrics.compute(ignore_case = True, ignore_numbers = True)


ValueError encountered: invalid literal for int() with base 10: 'DNA testing has for the first time confirmed the identity of the bacteria'
ValueError encountered: invalid literal for int() with base 10: 'there’s not much margin for error'
ValueError encountered: invalid literal for int() with base 10: '"The care that I was shown is second to none'
ValueError encountered: invalid literal for int() with base 10: 'Your Post'
ValueError encountered: invalid literal for int() with base 10: 'Inspired by the surface of a sponge'
ValueError encountered: invalid literal for int() with base 10: 'Sept. 9. 1'
ValueError encountered: invalid literal for int() with base 10: 'thieves to work out in what order the pattern was drawn'
ValueError encountered: invalid literal for int() with base 10: "Pebble's Core"
ValueError encountered: invalid literal for int() with base 10: 'no matter how many hours a day are spent sitting'
ValueError encountered: invalid literal for int() with base 10: 'acetaminophe

TypeError: F1._compute() got an unexpected keyword argument 'ignore_case'

In [1]:
tp = 0
tn = 0
fp = 0
fn = 0

dataloader = DataLoader(test_datasets, batch_size=32, shuffle = True)
evaluator = evaluate.evaluator("question-answering")
metrics = evaluate.combine(["exact_match", "f1"])

predicted = qa_pipelines[1](question=test_dataset["question"][1],
                            context=test_dataset["context"][1])
references = test_dataset["answers"][1]
print(predicted["answer"])
print(references)
# metrics.add(predicted["answer"], references[0])
# Add batch to metrics
try:
    results = evaluator.compute(
      model_or_pipeline=qa_pipelines[1],
      data=test_dataset,
      metric="exact_match"
  )
    #metrics.compute(ignore_case = True, ignore_numbers = True)
except ValueError as e:
    print("ValueError encountered:", e)
except Exception as e:
    print("An error occurred:", e)



NameError: name 'DataLoader' is not defined

In [52]:
#dataloader = DataLoader(test_ds, batch_size=16, shuffle = True)
metrics = evaluate.combine(["exact_match", "f1"])
results = evaluator.compute(
    model_or_pipeline=qa_pipelines[1],
    data=test_dataset,
    metric="exact_match"
)

# for list indices must be int and not string, the internet says turn it ito a dict??

Filter:   0%|          | 0/825 [00:00<?, ? examples/s]

TypeError: list indices must be integers or slices, not str

#### Evaluation

In [ ]:
def run_eval_test_qa(model, test_data_loader):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    total_correct_start = 0
    total_correct_end = 0
    total_correct_answers = 0
    total_samples = 0

    with torch.no_grad():
        for batch in test_data_loader:
            # print(len(batch))
            # print(batch['start_positions'][0])
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)
            spoiler = batch['answer'].to(device) # jel ovo uopce imamo u batchu? Mislim da ne?

            start_logits, end_logits = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            predictions = post_process_qa(start_logits, end_logits)
            start_predictions = predictions[0]
            end_predictions = predictions[1]
            answer_predictions = predictions[2]

            loss = qa_loss(start_predictions, end_predictions, start_positions, end_positions) # KOJI JE OVO LOSS

            total_correct_start += (start_predictions == start_positions).sum().item()
            total_correct_end += (end_predictions == end_positions).sum().item()
            total_correct_answers += (answer_predictions == spoiler).sum().item()
            total_samples += input_ids.shape[0]

    accuracy = total_correct_answers / total_samples
    print(f'Test Accuracy: {accuracy:.4f}')

### Summarization

#### Pre-processing functions

In [ ]:
def clean_dataset_sum(dataset):
  # remove multi answer entries and preprocess
  dataset = dataset.filter(lambda x: "multi" not in x["tags"] and x["targetTitle"] != "" and x["targetParagraphs"] != [])
  article = []
  start_positions = []
  end_positions = []
  for questions, paragraphs, answer, answer_positions in zip(dataset['targetTitle'], dataset['targetParagraphs'], dataset['targetTitle'], dataset["spoilerPositions"]):
        # reindex from [paragraph, index_in_paragraph] to index in total context
        # as context needs to be one string and not a list of strings
        start_index, end_index = recalculate_indices(paragraphs, answer, answer_positions)
        start_positions.append(start_index)
        end_positions.append(end_index)

        # join context as it can't be a list of strings/paragrapgh but just one string
        sentence = [questions]
        for i, paragraph in enumerate(paragraphs):
            sentence.append(str(i) + ": " + paragraph)
        article.append(" ".join(sentence))


  new_dataset = dataset.select_columns(['uuid', 'spoiler', 'tags'])

  new_dataset = new_dataset.rename_columns({'uuid': 'id', 'spoiler': 'answers'})
  print(type(new_dataset))
  dset_embed = datasets.Dataset.from_dict({"article":article, "start_positions":start_positions, "end_positions":end_positions})
  new_dataset = datasets.concatenate_datasets([new_dataset, dset_embed], axis=1)
  return new_dataset

In [ ]:
model_name = sum_model_names[0]
# a) Get predictions
book_sum_pipeline = pipeline('summarization', model=model_name, tokenizer=model_name)
ARTICLE = """Why is model conversion important? The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks."""
res = book_sum_pipeline(ARTICLE, max_length=10, min_length=3,no_repeat_ngram_size=3, encoder_no_repeat_ngram_size=3,)
print(res)
# b) Load model & tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
predicted = []
total_loss = 0
correct = 0
total = 0
for example in filtered:
    predicted = sum_pipelines[1](example["targetTitle"],context=example["targetPargraphs"])
    #predicted looks like {'score': 0.9041663408279419, 'start': 11, 'end': 18, 'answer': 'Philipp'}
    if predicted['answer'] == example['answer']:
        correct += 1
        total += 1
    # or should i use my loss?
    loss = F.cross_entropy(start_logits, example["start_positions"]) + F.cross_entropy(end_logits, example["end_positions"])
    total_loss += loss

accuracy = correct / total
loss = total_loss / len(filtered)
# TODO more measures

print(f'Accuracy: {accuracy:.4f}')
print(f'Loss: {loss:.4f}')


#### Evaluation

## Fine tuneing models

### Question answering

#### Training

#### Tokenization: copied from [here](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/question_answering-tf.ipynb#scrollTo=-b70jh26IrJS)


In [42]:
def prepare_train_features(examples, tokenizer = qa_tokenizers[1]):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=512,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # already did the reindexing?? But keep it here in case it turns out i needed this?

    # # Let's label those examples!
    # tokenized_examples["start_positions"] = []
    # tokenized_examples["end_positions"] = []

    # for i, offsets in enumerate(offset_mapping):
    #     # We will label impossible answers with the index of the CLS token.
    #     input_ids = tokenized_examples["input_ids"][i]
    #     cls_index = input_ids.index(tokenizer.cls_token_id)

    #     # Grab the sequence corresponding to that example (to know what is the context and what is the question).
    #     sequence_ids = tokenized_examples.sequence_ids(i)

    #     # One example can give several spans, this is the index of the example containing this span of text.
    #     sample_index = sample_mapping[i]
    #     answers = examples["answers"][sample_index]
    #     # If no answers are given, set the cls_index as answer.
    #     if len(answers["answer_start"]) == 0:
    #         tokenized_examples["start_positions"].append(cls_index)
    #         tokenized_examples["end_positions"].append(cls_index)
    #     else:
    #         # Start/end character index of the answer in the text.
    #         start_char = answers["answer_start"][0]
    #         end_char = start_char + len(answers["text"][0])

    #         # Start token index of the current span in the text.
    #         token_start_index = 0
    #         while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
    #             token_start_index += 1

    #         # End token index of the current span in the text.
    #         token_end_index = len(input_ids) - 1
    #         while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
    #             token_end_index -= 1

    #         # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
    #         if not (
    #             offsets[token_start_index][0] <= start_char
    #             and offsets[token_end_index][1] >= end_char
    #         ):
    #             tokenized_examples["start_positions"].append(cls_index)
    #             tokenized_examples["end_positions"].append(cls_index)
    #         else:
    #             # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
    #             # Note: we could go after the last offset if the answer is the last word (edge case).
    #             while (
    #                 token_start_index < len(offsets)
    #                 and offsets[token_start_index][0] <= start_char
    #             ):
    #                 token_start_index += 1
    #             tokenized_examples["start_positions"].append(token_start_index - 1)
    #             while offsets[token_end_index][1] >= end_char:
    #                 token_end_index -= 1
    #             tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [43]:
features = prepare_train_features(train_dataset[:5], qa_tokenizers[1])

In [50]:
tokenized_train_dataset = train_dataset.map(
    prepare_train_features, batched=True, remove_columns=train_dataset.column_names
)
tokenized_valid_dataset = valid_dataset.map(
    prepare_train_features, batched=True, remove_columns=valid_dataset.column_names
)

Map:   0%|          | 0/656 [00:00<?, ? examples/s]

#### Fine tuning

In [58]:
from transformers import TrainingArguments
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    metric_for_best_model="f1",

)

In [52]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [60]:
from transformers import Trainer
trainer = Trainer(
    model=qa_models[1],
    args=training_args,
    compute_metrics=qa_loss,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
    data_collator=data_collator
)

output = trainer.train()


TypeError: Trainer.__init__() got an unexpected keyword argument 'compute_loss'

In [61]:
loss = qa_loss(output['start_logits'], output['end_logits'], batch['start_positions'], batch['end_positions'])
loss.backward()
optimizer.step()

NameError: name 'output' is not defined

#### Evaluation

In [1]:
results = trainer.evaluate()
print(f"Evaluation Results: {results}")

NameError: name 'trainer' is not defined

Train loop

In [ ]:
for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    start_logits = []
    end_logits = []
    accelerator.print("Evaluation!")
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            outputs = model(**batch)

        start_logits.append(accelerator.gather(outputs.start_logits).cpu().numpy())
        end_logits.append(accelerator.gather(outputs.end_logits).cpu().numpy())

    start_logits = np.concatenate(start_logits)
    end_logits = np.concatenate(end_logits)
    start_logits = start_logits[: len(validation_dataset)]
    end_logits = end_logits[: len(validation_dataset)]

    metrics = compute_metrics(
        start_logits, end_logits, validation_dataset, raw_datasets["validation"]
    )
    print(f"epoch {epoch}:", metrics)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

In [ ]:
def post_process_qa(start_logits, end_logits, contexts):# Get the start and end logits
    # Get the top predictions
    start_indices = torch.argmax(start_logits, dim=1) # jel uzmemo stvarno samo jedan najvjerojatniji ili isprobati nekoliko opcija?
    end_indices = torch.argmax(end_logits, dim=1)

    # Filter out invalid predictions
    answers = []
    for i, start, end in enumerate(zip(start_indices, end_indices)):
        if start <= end:
            # find the answer in context:
            answers.append(contexts[i][start:end]) # jel ukljucujuci end?
        else:
            answers.append("No Answer") # u pravom formatu kako se pronalazi u datasetu
    return start_indices, end_indices, answers

In [ ]:
question_column_name = "question"
    context_column_name = "context"
    answer_column_name = "answers"

def prepare_validation_features(examples):
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_length,
            stride=data_args.doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length" if data_args.pad_to_max_length else False,
        )
        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0

            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])

            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]

        return tokenized_examples
data_collator = (
        default_data_collator
        if data_args.pad_to_max_length
        else DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8 if training_args.fp16 else None)
    )

    # Post-processing:
    def post_processing_function(examples, features, predictions, stage="eval"):
        # Post-processing: we match the start logits and end logits to answers in the original context.
        predictions = postprocess_qa_predictions(
            examples=examples,
            features=features,
            predictions=predictions,
            version_2_with_negative=data_args.version_2_with_negative,
            n_best_size=data_args.n_best_size,
            max_answer_length=data_args.max_answer_length,
            null_score_diff_threshold=data_args.null_score_diff_threshold,
            output_dir=training_args.output_dir,
            is_world_process_zero=trainer.is_world_process_zero(),
            prefix=stage,
        )
        # Format the result to the format the metric expects.
        if data_args.version_2_with_negative:
            formatted_predictions = [
                {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
            ]
        else:
            formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]

        references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]
        return EvalPrediction(predictions=formatted_predictions, label_ids=references)

trainer = QuestionAnsweringTrainer(
        model=model,
        args=training_args,
        train_dataset=None,
        eval_dataset=None,
        eval_examples=None,
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
        compute_metrics=lambda p: 0,
    )
return [{'uuid': i['id'], 'spoiler': i['prediction_text'].strip()} for i in trainer.predict(predict_dataset, predict_examples).predictions]

### Summarization

#### Pre-processing

#### Training

#### Evaluation

## Comparison